# Loan Application Study

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusion of Analysis</a></li>
<li><a href="#modeling">Modeling</a></li>
</ul>

<a id='intro'></a>
## Introduction
> **Business Overview:**
The business revolves around providing financial assistance to individuals or businesses in the form of loans. When someone applies for a loan, they are requesting a specific amount of money to be lent to them for a defined purpose. The lending institution evaluates the applicant's creditworthiness, risk profile, and financial stability to make informed decisions about granting or denying the loan.
>
> **Problem Statement:**
> The problem is to predict whether a loan will be approved or not based on the applicant's information.
> 
> **Data set:** You can find it [here](https://www.kaggle.com/datasets/gauravduttakiit/loan-defaulter?select=application_data.csv) on Kaggle. Consists of two files: current application data and previous applications data.
>
> **Strategy of using both datasets:**
> I will use the previous dataset only to get aggregated information about the applicant's history. Then I will merge it with the current application data to get the final dataset for analysis.

> ### Questions for Analysis we need to answer.
>
> **Applicant's info**
> 1. How does the clients' Demographics such as the age related to the approval?
> 2. Is there a relation between income and the approval of loan?
> 3. How does the type of work or housing sitiuation related to the approval?
>
> **Application's info**
> 1. 

## Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

curr_df = pd.read_csv('./Dataset/application_data.csv')
prev_df = pd.read_csv('./Dataset/previous_application.csv')

curr_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


> Making the names of the columns in lower case for simplicity

In [2]:
curr_lower_case_columns_dict = {column: column.lower() for column in curr_df.columns}
prev_lower_case_columns_dict = {column: column.lower() for column in prev_df.columns}

curr_df.rename(columns=curr_lower_case_columns_dict, inplace=True)
prev_df.rename(columns=prev_lower_case_columns_dict, inplace=True)

curr_df.head(1)

,sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,...,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


### Data Assessing

> Lets start with prev_df to get aggregated insights from it

In [19]:
prev_df.groupby(['sk_id_curr', 'sk_id_prev']).count()['name_contract_type']

sk_id_curr  sk_id_prev
100001      1369693       1
100002      1038818       1
100003      1810518       1
            2396755       1
            2636178       1
                         ..
456255      1708056       1
            1743609       1
            2073384       1
            2631384       1
            2729207       1
Name: name_contract_type, Length: 1670214, dtype: int64

> There are many current applications that have previous applications.
> 
> For this reason we will aggregate the previous applications data to get the insights we need and then merge it with the current applications data.

In [29]:
# Dataframe we will save our aggregated data to
agg_df = pd.DataFrame()

> Calculating the avg amout of credit taken by the applicant

In [33]:
agg_df['avg_credit_taken'] = prev_df.groupby('sk_id_curr').mean()['amt_credit']

> Getting the status of the previous applications based on approval rate and rejection rate.

In [74]:
grouped_data_approval_refusion = prev_df.groupby(['sk_id_curr', 'name_contract_status']).count().unstack(fill_value=0)['sk_id_prev']

agg_df['approval_rate'] = grouped_data_approval_refusion['Approved'] / grouped_data_approval_refusion.sum(axis=1)
agg_df['rejection_rate'] = grouped_data_approval_refusion['Refused'] / grouped_data_approval_refusion.sum(axis=1)

def get_previous_status(row, threshold=0.1):
    """
    Returns the previous status as 'Good' or 'Bad' based on the approval and rejection rates
    Inputs:
        row: A row of the dataframe
        threshold: The threshold to be used to determine if the previous status was good or bad
    
    Returns:
        1: If the previous status was good
        -1: If the previous status was bad
        0: If the previous status was neutral
    """
    
    if (row.approval_rate - row.rejection_rate) > threshold:
        return 1
    elif (row.rejection_rate - row.approval_rate) > threshold:
        return -1
    else:
        return 0

agg_df['prev_status'] = agg_df.apply(lambda row: get_previous_status(row), axis=1)
agg_df['prev_status'].value_counts()

 1    293656
 0     25086
-1     20115
Name: prev_status, dtype: int64

In [77]:
agg_df.query('prev_status == 0').head()

,avg_credit_taken,approval_rate,rejection_rate,prev_status
sk_id_curr,,,,
100035,161788.295455,0.363636,0.363636,0
100046,481817.250000,0.500000,0.500000,0
100051,50689.500000,0.333333,0.333333,0
100072,219881.700000,0.300000,0.200000,0
100078,422988.136364,0.363636,0.363636,0


In [27]:

agg_df['number_of_rejections'] = 

sk_id_curr
100001     23787.00
100002    179055.00
100003    484191.00
100004     20106.00
100005     20076.75
            ...    
456251     40455.00
456252     56821.50
456253     20625.75
456254    134439.75
456255    424431.00
Name: amt_credit, Length: 338857, dtype: float64

In [30]:
aggregator = prev_df.groupby(['sk_id_curr'])
(prev_df.groupby(['sk_id_curr']).count()['sk_id_prev'] == 1).mean()

0.17841744452674727

In [3]:
curr_df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Non-Null Count   Dtype  
---   ------                        --------------   -----  
 0    sk_id_curr                    307511 non-null  int64  
 1    target                        307511 non-null  int64  
 2    name_contract_type            307511 non-null  object 
 3    code_gender                   307511 non-null  object 
 4    flag_own_car                  307511 non-null  object 
 5    flag_own_realty               307511 non-null  object 
 6    cnt_children                  307511 non-null  int64  
 7    amt_income_total              307511 non-null  float64
 8    amt_credit                    307511 non-null  float64
 9    amt_annuity                   307499 non-null  float64
 10   amt_goods_price               307233 non-null  float64
 11   name_type_suite               306219 non-null  object 
 12   name_income_type            

> There are many null values in the dataset, we need to check if they are important or not.

In [4]:
print(f'Number of duplicates: {curr_df.duplicated().sum()}')

Number of duplicates: 0


In [6]:
curr_df.select_dtypes(np.number).describe()

,sk_id_curr,target,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,region_population_relative,days_birth,days_employed,...,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


> * 